In [ ]:
#r "nuget: Microsoft.Azure.CognitiveServices.Vision.Face, 2.8.0-preview.3"

In [8]:
 // Import namespaces
using System.IO;
using System.Drawing;

using Microsoft.Azure.CognitiveServices.Vision.Face;
using Microsoft.Azure.CognitiveServices.Vision.Face.Models;

In [13]:
string cogSvcEndpoint = "YOUR_AZURE_AI_SERVICES_ENDPOINT";
string cogSvcKey = "YOUR_AZURE_AI_SERVICES_KEY";

// string imageFile = "C-Sharp/face-api/images/people.jpg";
string imageFile = "C-Sharp/face-api/images/people2.jpg";
// string imageFile = "C-Sharp/face-api/images/person1.jpg";
// string imageFile = "C-Sharp/face-api/images/person2.jpg";


In [14]:
// Authenticate Face client
ApiKeyServiceClientCredentials credentials = new ApiKeyServiceClientCredentials(cogSvcKey);
FaceClient client = new FaceClient(credentials)
{
    Endpoint = cogSvcEndpoint
};

In [15]:
static async Task DetectFaces(string imageFile, FaceClient client)
{
    Console.WriteLine($"Detecting faces in {imageFile}");

    // Specify facial features to be retrieved
    IList<FaceAttributeType> features = new FaceAttributeType[]
    {
        FaceAttributeType.Occlusion,
        FaceAttributeType.Blur,
        FaceAttributeType.Glasses
    };

    // Get faces
    // Get faces
    using (var imageData = File.OpenRead(imageFile))
    {    
        var detected_faces = await client.Face.DetectWithStreamAsync(imageData, returnFaceAttributes: features, returnFaceId: false);

        if (detected_faces.Count() > 0)
        {
            Console.WriteLine($"{detected_faces.Count()} faces detected.");

            // Prepare image for drawing
            Image image = Image.FromFile(imageFile);
            Graphics graphics = Graphics.FromImage(image);
            Pen pen = new Pen(Color.LightGreen, 3);
            Font font = new Font("Arial", 4);
            SolidBrush brush = new SolidBrush(Color.White);
            int faceCount=0;

            // Draw and annotate each face
            foreach (var face in detected_faces)
            {
                faceCount++;
                Console.WriteLine($"\nFace number {faceCount}");
                
                // Get face properties
                Console.WriteLine($" - Mouth Occluded: {face.FaceAttributes.Occlusion.MouthOccluded}");
                Console.WriteLine($" - Eye Occluded: {face.FaceAttributes.Occlusion.EyeOccluded}");
                Console.WriteLine($" - Blur: {face.FaceAttributes.Blur.BlurLevel}");
                Console.WriteLine($" - Glasses: {face.FaceAttributes.Glasses}");

                // Draw and annotate face
                var r = face.FaceRectangle;
                Rectangle rect = new Rectangle(r.Left, r.Top, r.Width, r.Height);
                graphics.DrawRectangle(pen, rect);
                string annotation = $"Face number {faceCount}";
                graphics.DrawString(annotation,font,brush,r.Left, r.Top);
            }

            // Save annotated image
            String output_file = "detected_faces.jpg";
            image.Save(output_file);
            Console.WriteLine(" Results saved in " + output_file);   
        }
    }
}

In [ ]:
await DetectFaces(imageFile, client);